In [1]:
# Import necessary module
from sqlalchemy import create_engine
import pandas as pd

# Create engine: engine
engine = create_engine('sqlite:///programming.sqlite')

In [2]:
# What are the tables in the database?

# Save the table names to a list: table_names
table_names  = engine.table_names()

# Print the table names to the shell
print(table_names)

# See the data model here: http://chinookdatabase.codeplex.com/wikipage?title=Chinook_Schema&referringTitle=Home


['Grade']


In [3]:
# The power of SQL lies in relationships between tables: INNER JOIN

# Execute query and store records in DataFrame: df
df = pd.read_sql_query("select * from Grade",engine)

# drop the unnecessary column
df = df.drop('index', 1)

# print the number of unique students
print(len(df['Aluno_ID'].unique()))

# Print head of DataFrame
df.head()


766


,Aluno_ID,Período,Disciplina,Situaçăo,Média Final,Unidade I,Unidade II,Unidade III
0,0,2014.1,IMD0012.0 - INTRODUÇĂO ŔS TÉCNICAS DE PROGRAMAÇĂO,APROVADO,7.0,4.9,9.0,7.0
1,1,2014.1,IMD0012.0 - INTRODUÇĂO ŔS TÉCNICAS DE PROGRAMAÇĂO,APROVADO,7.3,8.0,7.0,7.0
2,2,2014.1,IMD0012.0 - INTRODUÇĂO ŔS TÉCNICAS DE PROGRAMAÇĂO,APROVADO,9.3,9.5,8.3,10.0
3,3,2014.1,IMD0012.0 - INTRODUÇĂO ŔS TÉCNICAS DE PROGRAMAÇĂO,APROVADO,7.3,6.5,7.0,8.3
4,4,2014.1,IMD0012.0 - INTRODUÇĂO ŔS TÉCNICAS DE PROGRAMAÇĂO,APROVADO,7.5,5.5,8.0,9.0


In [19]:
semesters = pd.read_sql_query("SELECT DISTINCT Período FROM Grade", engine)

In [20]:
semesters = semesters[semesters.columns[0]].tolist()
semesters
first_semester = min(semesters)

In [12]:
maxID = pd.read_sql_query("SELECT MAX(Aluno_ID) as max FROM Grade", engine) 
maxID = maxID['max'][0]

In [63]:
students = dict()
for i in range(0, maxID):
    student = pd.read_sql_query(
        "SELECT Período, Disciplina, \"Situaçăo\", \"Média Final\" FROM Grade WHERE Aluno_ID = " + str(i), 
        engine)
    print(student)
    students[i] = student

   Período                                         Disciplina  \
0   2014.1  IMD0012.0 - INTRODUÇĂO ŔS TÉCNICAS DE PROGRAMAÇĂO   
1   2014.1    IMD0012.1 - PRÁTICAS DE TÉCNICAS DE PROGRAMAÇĂO   
2   2014.2             IMD0029 - ESTRUTURA DE DADOS BÁSICAS I   
3   2014.2               IMD0030 - LINGUAGEM DE PROGRAMAÇĂO I   
4   2015.1           IMD0039 - ESTRUTURAS DE DADOS BÁSICAS II   
5   2015.1              IMD0040 - LINGUAGEM DE PROGRAMAÇĂO II   

            Situaçăo  Média Final  
0           APROVADO          7.0  
1           APROVADO          9.2  
2           APROVADO          7.2  
3           APROVADO          7.9  
4  APROVADO POR NOTA          6.4  
5           APROVADO          8.7  
   Período                                         Disciplina  \
0   2014.1  IMD0012.0 - INTRODUÇĂO ŔS TÉCNICAS DE PROGRAMAÇĂO   
1   2014.1    IMD0012.1 - PRÁTICAS DE TÉCNICAS DE PROGRAMAÇĂO   
2   2015.1             IMD0029 - ESTRUTURA DE DADOS BÁSICAS I   
3   2015.1               IMD0030

In [169]:
studentsBySemester = dict()
for s in semesters:
    studentsBySemester[s] = []
for key, value in students.items():
    first = value[value['Disciplina'] == 'IMD0012.0 - INTRODUÇĂO ŔS TÉCNICAS DE PROGRAMAÇĂO']['Período']
    first = first.tolist()
    if len(first) > 0:
        first = first[0]
        studentsBySemester[first].append(value)

In [175]:
def studentGrades(grades):
    print(grades)
    courses = grades.loc[
        (grades['Situaçăo'] == "APROVADO") | 
        (grades['Situaçăo'] == "APROVADO POR NOTA")]
    
    s = 0
    count = 0
    itp = 0
    for index, row in courses.iterrows(): 
        if not "IMD0012.0" in row['Disciplina']:
            s += row['Média Final']
            count += 1
        else:
            itp = row['Média Final']
    if count > 0:
        average = s/count
    else:
        average = 0
    return itp, average

In [176]:
for key, value in studentsBySemester.items():
    points = []
    for grades in value:
        points.append([studentGrades(grades)])

   Período                                         Disciplina  \
0   2014.1  IMD0012.0 - INTRODUÇĂO ŔS TÉCNICAS DE PROGRAMAÇĂO   
1   2014.1    IMD0012.1 - PRÁTICAS DE TÉCNICAS DE PROGRAMAÇĂO   
2   2014.2             IMD0029 - ESTRUTURA DE DADOS BÁSICAS I   
3   2014.2               IMD0030 - LINGUAGEM DE PROGRAMAÇĂO I   
4   2015.1           IMD0039 - ESTRUTURAS DE DADOS BÁSICAS II   
5   2015.1              IMD0040 - LINGUAGEM DE PROGRAMAÇĂO II   

            Situaçăo  Média Final  
0           APROVADO          7.0  
1           APROVADO          9.2  
2           APROVADO          7.2  
3           APROVADO          7.9  
4  APROVADO POR NOTA          6.4  
5           APROVADO          8.7  
   Período                                         Disciplina  \
0   2014.1  IMD0012.0 - INTRODUÇĂO ŔS TÉCNICAS DE PROGRAMAÇĂO   
1   2014.1    IMD0012.1 - PRÁTICAS DE TÉCNICAS DE PROGRAMAÇĂO   
2   2015.1             IMD0029 - ESTRUTURA DE DADOS BÁSICAS I   
3   2015.1               IMD0030